In [1]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
data = pd.read_csv("SMSSpamCollection.tsv", sep="\t", header=None)
data.columns= ['label', 'body_text']
import string


In [12]:
data

,label,body_text,body_len,punct%
0,ham,I've been searching for the right words to thank you for this breather. I promise i wont take yo...,160,2.5
1,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,128,4.7
2,ham,"Nah I don't think he goes to usf, he lives around here though",49,4.1
3,ham,Even my brother is not like to speak with me. They treat me like aids patent.,62,3.2
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,28,7.1
...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy...,131,6.1
5564,ham,Will ü b going to esplanade fr home?,29,3.4
5565,ham,"Pity, * was in mood for that. So...any other suggestions?",48,14.6
5566,ham,The guy did some bitching but I acted like i'd be interested in buying something else next week ...,100,1.0


In [13]:
data ['label']= np.where(data['label']=='ham',0, 1)

In [14]:
data

,label,body_text,body_len,punct%
0,0,I've been searching for the right words to thank you for this breather. I promise i wont take yo...,160,2.5
1,1,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,128,4.7
2,0,"Nah I don't think he goes to usf, he lives around here though",49,4.1
3,0,Even my brother is not like to speak with me. They treat me like aids patent.,62,3.2
4,0,I HAVE A DATE ON SUNDAY WITH WILL!!,28,7.1
...,...,...,...,...
5563,1,This is the 2nd time we have tried 2 contact u. U have won the £750 Pound prize. 2 claim is easy...,131,6.1
5564,0,Will ü b going to esplanade fr home?,29,3.4
5565,0,"Pity, * was in mood for that. So...any other suggestions?",48,14.6
5566,0,The guy did some bitching but I acted like i'd be interested in buying something else next week ...,100,1.0


In [15]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100
    
    
data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data["punct%"] = data['body_text'].apply(lambda x: count_punct(x))

In [16]:
#Используем функцию для удаления пунктуации токенизации и удаления стоп слов и стемминга
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [17]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)

In [18]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [22]:
rf = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
         'max_depth': [30, 60, 90, None]
}

In [28]:
X_tfidf_feat= X_tfidf_feat.astype(np.uint8)

In [29]:
import numpy as np
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(X_tfidf_feat, data['label'].astype(np.uint8))
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

MemoryError: Unable to allocate 34.4 MiB for an array with shape (8109, 4454) and data type uint8

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [21]:
y_train

3543    ham
462     ham
1108    ham
5193    ham
1306    ham
       ... 
4557    ham
4778    ham
5503    ham
4265    ham
2207    ham
Name: label, Length: 4454, dtype: object

In [17]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average= 'binary')
    print('Est: {}/ Depth: {} / ====== Precision: {} / Recall: {} / Accuracy: {}'.format(
    n_est, depth, round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)
    ))

In [18]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 20, None]:
        train_RF(n_est, depth)

Est: 10/ Depth: 10 / ====== Precision: 1.0 / Recall: 0.359 / Accuracy: 0.91
Est: 10/ Depth: 20 / ====== Precision: 1.0 / Recall: 0.628 / Accuracy: 0.948
Est: 10/ Depth: 20 / ====== Precision: 0.99 / Recall: 0.628 / Accuracy: 0.947
Est: 10/ Depth: None / ====== Precision: 0.992 / Recall: 0.814 / Accuracy: 0.973
Est: 50/ Depth: 10 / ====== Precision: 1.0 / Recall: 0.224 / Accuracy: 0.891
Est: 50/ Depth: 20 / ====== Precision: 1.0 / Recall: 0.635 / Accuracy: 0.949
Est: 50/ Depth: 20 / ====== Precision: 1.0 / Recall: 0.609 / Accuracy: 0.945
Est: 50/ Depth: None / ====== Precision: 0.993 / Recall: 0.872 / Accuracy: 0.981
Est: 100/ Depth: 10 / ====== Precision: 1.0 / Recall: 0.314 / Accuracy: 0.904
Est: 100/ Depth: 20 / ====== Precision: 1.0 / Recall: 0.609 / Accuracy: 0.945
Est: 100/ Depth: 20 / ====== Precision: 1.0 / Recall: 0.66 / Accuracy: 0.952
Est: 100/ Depth: None / ====== Precision: 0.985 / Recall: 0.853 / Accuracy: 0.978


In [27]:
'Precision:{}/ Recall: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3))

'Precision:1.0/ Recall: 0.588 / Accuracy: 0.943'